# Где собирать логи
1. Ставим Docker desktop
2. Устанавливаем [образ](https://hub.docker.com/r/yandex/clickhouse-server/) Clickhouse
```
!docker run -d -p 0.0.0.0:8123:8123 --volume=/path/to/some/folder/on/disk/some_clickhouse_database:/var/lib/clickhouse --name some-clickhouse-server --ulimit nofile=262144:262144 yandex/clickhouse-server
```

Опция -p 0.0.0.0:8123:8123 открывает доступ к контейнеру по порту 8123 (иногда сразу его нет).

При повторной установке в случае ошибки вида
```
docker: Error response from daemon: Conflict. The container name "/some-clickhouse-server" is already in use by container "34899ff1c1d78111048b762fb730963adac0b90eedb9751f4c5d62aa4d90c589". You have to remove (or rename) that container to be able to reuse that name.
```
удалите контейнер командой (только замените ID контейнера на свой)
```
!docker rm 34899ff1c1d78111048b762fb730963adac0b90eedb9751f4c5d62aa4d90c589
```

Как узнать ID_контейнера
```
!docker ps
```

Как зайти в контейнер (лучше делать в командной строке):
```
docker exec -it ID_контейнера bash
```

Открыть clickhouse-client:
```
docker run -it --rm --link some-clickhouse-server:clickhouse-server yandex/clickhouse-client --host clickhouse-server
```

3. Проверьте наличие доступа к clickhouse в контейнере в браузере, открыв ссылку [localhost:8123](http://localhost:8123), должны увидеть Ok.

4. Берем открытые данные [Метрики](https://clickhouse.tech/docs/ru/getting-started/example-datasets/metrica/).

In [1]:
import requests
from io import StringIO
import pandas as pd

In [2]:
HOST = 'http://localhost:8123'

In [3]:
def clickhouse_data(query, host=HOST, timeout=60, columns=None):
    r = requests.post(host, params = {'timeout_before_checking_execution_speed': 120, 'max_execution_time': 6000}
                          , timeout = timeout, data = query)
    if r.status_code == 200:
        return r.text
    else:
        print('Что-то пошло не так')
        raise ValueError(r.text)

In [18]:
clickhouse_data('select count(*) from datasets.hits_v1')

'8873898\n'

In [32]:
text = clickhouse_data('select BrowserCountry, count(*) as cnt from datasets.hits_v1 group by BrowserCountry order by cnt desc limit 10')
text

'��\t6048025\nTp\t2131666\nTi\t361055\nI7\t175930\nIP\t32667\nMu\t23502\nmD\t10737\nK6\t10410\nax\t8808\n4t\t7780\n'

In [33]:
df = pd.read_csv(StringIO(text), sep='\t', names=['BrowserCountry', 'cnt'])
df

,BrowserCountry,cnt
0,��,6048025
1,Tp,2131666
2,Ti,361055
3,I7,175930
4,IP,32667
5,Mu,23502
6,mD,10737
7,K6,10410
8,ax,8808
9,4t,7780


In [11]:
%%time

text = clickhouse_data('select EventDate, count(*) from datasets.hits_v1 group by EventDate order by EventDate')
df = pd.read_csv(StringIO(text), sep='\t', names=['EventDate', 'hits'])

CPU times: user 5.62 ms, sys: 1.72 ms, total: 7.34 ms
Wall time: 40.3 ms


In [31]:
df

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,BrowserCountry,cnt
7043438415214026105,1,NaN,1,2014-03-17 15:29:13,2014-03-17,57,706706104,��:�[�Uc��m���,14050,8585742290196126178,0,56,4,"http://hurpasspx?Entitle,pos,p5,source=wiz&fp=2&pos=42&wp=1&pos=7&lang=ru&lr=10275961,55.84517842040/?fromServic-last/moto.am/iadebnye_uchasti-shedshipmeethodID.asp&is_vative=0&search-tag=2&rpt=simages.yandex.ru/rp/semennoe/zarplatjie-babe-podpisyvat-otvety.ru/tour=170000009886%26ref%3Dhttp://kryma-parca.ru/chat&users/lexus-70@mail.yandex.ru/yarportwo/11868/parta.com/load/483/?from]=200837&tease",http://kelebekgaleri.hurriyet,hurpass.uaprod.hu,sprashivai.ru,0,0,"[5,353,3121,11503,11630]","[5,92,469,13425]","[348,1010]","[28,644]",1234,573,29,8,0,0.0,0,0,44,s�,1,1,0,0,NaN,NaN,1117402,1,0,NaN,0,0,266,111,117,2014-03-16 00:15:16,0,0,0,0,utf-8,510,0,0,0,6595113484233243093,229156632,0,0,0,0,0,E,2014-03-17 07:48:07,39,1,2,0,0,[],594601839,���_Y����5Ӵ.,6826,-1,1,nD,��,NaN,NaN,0,-1,139,175,197,11,0,0,798,-1,-1,-1,-1,-1,-1,-1,0,NaN,0,NaN,�,0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5301774355650737480,2999314218052725538,0,0,NaN,NaN,NaN,[],[],[],[],[],[],���+������bKQ9,104,0
4944118417295196513,1,вышивка в Москве - Образовать фото мочия - Почта Mail.Ru: Силва,1,2014-03-17 09:19:26,2014-03-17,57,1581336367,��:�[�Uc��m���,54,610708775678702928,0,1,3,"http://hurpasspx?EntityType=images.jpg,http:%2F%2Fweb%2Fimages",http://iconnId=140Z1BmWE9JVEdoQ2Zud01aa0f8b72a2cb141ad2fbb6bc0488a293f1c0b7bbfe6e0921396325_7860c5b30e0216&mb_url=http://svit/realesta.com/?do=news.ru/yandex.php?thread%2FEl0TTQQZIClEGYFcJG1F4XldSeWtvVkFkf38xIAEsQVgWCVtUe15_d34cemhbU0dIfji-RM,hurpass.uaprod.hu,incityadspix.com.vn,0,0,"[6,98,456,8586]","[5,92,469,13425]","[348,1010]","[267,694]",1846,952,29,8,0,0.0,0,0,24,s�,1,1,0,0,NaN,NaN,3223479,-1,0,NaN,0,1,1246,906,157,2014-03-17 12:32:09,5,1,19832,0,utf-8,291,0,0,0,5970711023083247428,117479153,0,0,0,1,0,E,2014-03-17 09:03:08,55,1,2,16014,8,"[5,2,14,925,4,3993,72,6,9,27,1,3,1019]",2024189568,���_Y����5Ӵ.,8687,-1,1,nD,Tp,NaN,NaN,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,7557,-1,-1,0,NaN,0,NaN,�,0,[834016],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7613536979754271517,966296541083783832,0,0,NaN,NaN,NaN,[],[],[],[],[],[],���+������bKQ9,1324,1
4675118455796596620,1,вышивка в Москве - Образовать фото мочия - Почта Mail.Ru: Силва,1,2014-03-17 09:19:26,2014-03-17,57,1581336367,��:�[�Uc��m���,54,610708775678702928,0,1,3,http://webmodelka/118820005260808&cond=onair.html%3Fbprtment-arizm.ru/task/12810faSOWQ%3D%3DfrSMw%3D%3DfsSaHR0cDovL21lbWJlcmxpYWV0c2tv.ru/s/21/03/20/region;noagent.do?NextServingatlanakomster-dzhemchura.ru%2fcity=on&default-summessage/2229000012173/tuk196EII5dZrmAg,NaN,hurpass.uaprod.hu,NaN,0,0,[],"[5,92,469,13425]","[348,1010]",[],1846,952,29,8,0,0.0,0,0,24,s�,1,1,0,0,NaN,NaN,3223479,0,0,NaN,0,0,1246,906,157,2014-03-17 12:32:09,5,1,19832,0,utf-8,291,0,0,0,5970711023083247428,117479153,0,0,0,0,0,E,2014-03-17 09:03:08,55,1,2,16014,8,"[5,2,14,925,4,3993,72,6,9,27,1,3,1019]",2024189568,���_Y����5Ӵ.,8687,-1,1,nD,Tp,NaN,NaN,0,-1,0,0,1959,389,439,0,2414,-1,-1,-1,-1,-1,-1,-1,0,NaN,0,NaN,�,0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,15284527577228392792,7715869906126080954,0,0,NaN,NaN,NaN,[],[],[],[],[],[],���+������bKQ9,1146,0
5996666368919599438,1,вышивка в Москве - Образовать фото мочия - Почта Mail.Ru: Силва,1,2014-03-17 04:38:52,2014-03-17,57,1581336367,��:�[�Uc��m���,54,610708775678702928,0,1,3,http://webmodelka/118820005260808&cond=onair.html%3Fbprtment-arizm.ru/task/12810faSOWQ%3D%3DfrSMw%3D%3DfsSaHR0cDovL21lbWJlcmxpYWV0c2tv.ru/s/21/03/20/region;noagent.do?NextServingatlanakomster-dzhemchura.ru%2fcity=on&default-summessage/2229000012173/tuk196EII5dZrmAg,NaN,hurpass.uaprod.hu,NaN,0,0,[],"[5,92,469,13425]","[348,1010]",[],1846,952,29,8,0,0.0,0,0,24,s�,1,1,0,0,NaN,NaN,3223479,0,0,NaN,0,0,1246,906,157,2014-03-17 07:19:00,5,1,

In [13]:
count(distinct UserID)

SyntaxError: invalid syntax (2927307047.py, line 1)

In [14]:
text = clickhouse_data('select EventDate, uniq(UserID) from datasets.hits_v1 group by EventDate order by EventDate')
df = pd.read_csv(StringIO(text), sep='\t', names=['EventDate', 'unique_users_approx'])
df

,EventDate,unique_users_approx
0,2014-03-17,36613
1,2014-03-18,36531
2,2014-03-19,36940
3,2014-03-20,36462
4,2014-03-21,35447
5,2014-03-22,31555
6,2014-03-23,31200


In [15]:
text = clickhouse_data('select EventDate, uniqExact(UserID) from datasets.hits_v1 group by EventDate order by EventDate')
df = pd.read_csv(StringIO(text), sep='\t', names=['EventDate', 'unique_users_exact'])
df

,EventDate,unique_users_exact
0,2014-03-17,36613
1,2014-03-18,36531
2,2014-03-19,36940
3,2014-03-20,36462
4,2014-03-21,35447
5,2014-03-22,31555
6,2014-03-23,31200


In [16]:
%%time
text = clickhouse_data('select TraficSourceID, EventDate, uniqExact(UserID) from datasets.hits_v1 group by TraficSourceID, EventDate')
df = pd.read_csv(StringIO(text), sep='\t', names=['TraficSourceID', 'EventDate', 'unique_users_exact'])

CPU times: user 6.02 ms, sys: 2.04 ms, total: 8.06 ms
Wall time: 265 ms


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   TraficSourceID      70 non-null     int64 
 1   EventDate           70 non-null     object
 2   unique_users_exact  70 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ KB
